In [ ]:
%%writefile Dockerfile
# ffmpeg build - simply change versions to compile
# This Dockerfile is based on docker image  rottenberg/ffmpeg
# The purpose behind this is to build ffmpeg inside a debian container.
# 	Then the next step is to export the compiled binaries into an untouched base debian image
# 	This results in a ~50% images size reduction.
# 	FYI:  this image is useable....I just like having a smaller base image to download.
# Some of these options I don't use, so I commented them out.
# My builds include only FFMPEG + libfdk_aac + latest x264 + Decklink(Blackmagic)
# I am including Decklink(Blackmagic) so I can utilize those devices
# I don't need anything else.  If you need anything included, email me and I can make alternative builds.
# I will be tracking the 'stable' rolling release of Debian

FROM		ubuntu:18.04
MAINTAINER	jack northrup <madhatterstamps@gmail.com>

ENV	FFMPEG_VERSION		3.2.4
	# monitor releases at https://github.com/FFmpeg/FFmpeg/releases
ENV	YASM_VERSION    	1.3.0
	# monitor releases at https://github.com/yasm/yasm/releases
ENV	FDKAAC_VERSION  	0.1.5
	# monitor releases at https://github.com/mstorsjo/fdk-aac/releases
#ENV	x264
	# this project does not use release versions at this time
	# monitor project at http://git.videolan.org/?p=x264.git;a=shortlog
#ENV	LAME_VERSION    	3.99.5
#ENV	FAAC_VERSION    	1.28
#ENV	XVID_VERSION    	1.3.3
#ENV	MPLAYER_VERSION 	1.1.1

ENV	SRC             	/usr/local
ENV	LD_LIBRARY_PATH 	${SRC}/lib
ENV	PKG_CONFIG_PATH 	${SRC}/lib/pkgconfig

RUN bash -c 'set -euo pipefail'
RUN apt-get update
RUN apt-get install -y autoconf automake gcc build-essential git libtool make nasm zlib1g-dev tar curl

# YASM
RUN DIR=$(mktemp -d) && cd ${DIR} && \
              curl -Os http://www.tortall.net/projects/yasm/releases/yasm-${YASM_VERSION}.tar.gz && \
              tar xzvf yasm-${YASM_VERSION}.tar.gz && \
              cd yasm-${YASM_VERSION} && \
              ./configure --prefix="$SRC" --bindir="${SRC}/bin" && \
              make && \
              make install && \
              make distclean && \
              rm -rf ${DIR}

# x264
RUN DIR=$(mktemp -d) && cd ${DIR} && \
              git clone --depth 1 git://git.videolan.org/x264 && \
              cd x264 && \
              ./configure --prefix="$SRC" --bindir="${SRC}/bin" --enable-static && \
              make && \
              make install && \
              make distclean && \
              rm -rf ${DIR}

# LAME
#RUN DIR=$(mktemp -d) && cd ${DIR} && \
#              curl -L -Os http://downloads.sourceforge.net/project/lame/lame/${LAME_VERSION%.*}/lame-${LAME_VERSION}.tar.gz  && \
#              tar xzvf lame-${LAME_VERSION}.tar.gz  && \
#              cd lame-${LAME_VERSION} && \
#              ./configure --prefix="${SRC}" --bindir="${SRC}/bin" --disable-shared --enable-nasm && \
#              make && \
#              make install && \
#              make distclean && \
#              rm -rf ${DIR}

# FAAC
	# This combines faac + http://stackoverflow.com/a/4320377
RUN DIR=$(mktemp -d) && cd ${DIR} && \
              curl -L -Os http://downloads.sourceforge.net/faac/faac-${FAAC_VERSION}.tar.gz  && \
              tar xzvf faac-${FAAC_VERSION}.tar.gz  && \
              cd faac-${FAAC_VERSION} && \
              sed -i '126d' common/mp4v2/mpeg4ip.h && \
              ./bootstrap && \
              ./configure --prefix="${SRC}" --bindir="${SRC}/bin" && \
              make && \
              make install &&\
              rm -rf ${DIR}

# XVID
RUN DIR=$(mktemp -d) && cd ${DIR} && \
              curl -L -Os  http://downloads.xvid.org/downloads/xvidcore-${XVID_VERSION}.tar.gz  && \
              tar xzvf xvidcore-${XVID_VERSION}.tar.gz && \
              cd xvidcore/build/generic && \
              ./configure --prefix="${SRC}" --bindir="${SRC}/bin" && \
              make && \
              make install && \
              rm -rf ${DIR}

# FDK_AAC
RUN DIR=$(mktemp -d) && cd ${DIR} && \
              curl -s https://codeload.github.com/mstorsjo/fdk-aac/tar.gz/v${FDKAAC_VERSION} | tar zxvf - && \
              cd fdk-aac-${FDKAAC_VERSION} && \
              autoreconf -fiv && \
              ./configure --prefix="${SRC}" --disable-shared && \
              make && \
              make install && \
              make distclean && \
              rm -rf ${DIR}

# FFMPEG
	# I removed these flags from configure:  --enable-libfaac --enable-libmp3lame  --enable-libxvid
RUN DIR=$(mktemp -d) && cd ${DIR} && \
              curl -Os http://ffmpeg.org/releases/ffmpeg-${FFMPEG_VERSION}.tar.gz && \
              tar xzvf ffmpeg-${FFMPEG_VERSION}.tar.gz 

RUN apt install -y \
DEBIAN_FRONTEND=noninteractive apt-get install -y \
    x11vnc \
    xvfb \
    xterm \
    fluxbox \
frei0r-plugins frei0r-plugins-dev gnome-video-effects-frei0r libass-dev libass-devel \
libass-dev libdc1394-22-dev libmp3lame-dev libmp3lame0 libsox-fmt-mp3 libtheora-dev \
libtheora-bin libtheora0 libvorbis-dev libvpx-dev libwavpack-dev libwavpack1 \
libx264-152 libx264-dev libx265-146 libx265-dev x265 libx265-146 libx265-dev x265 \
libxvidcore-dev libxvidcore4 libopenjp2-7-dev libopenjp2-7 libopus-dev librtmp-dev librtmp1 \
libsoxr-dev libsoxr-lsr0 libsoxr0 speex libspeex-dev libvo-amrwbenc-dev libwebp-dev libwebp6 \
libespeak-dev libespeak1 libav-tools python3-pyopencl libclc-dev pulseaudio pulseaudio-dev 
RUN apt install -y \
youtube-dl winff libavcodec-extra libavcodec-extra57 libvo-aacenc-dev vlc-plugin-access-extra \
meld rubberband-ladspa libiec61883-dev ladspa-sdk libopencv-apps-dev libopencv-apps0d \
libopencv-calib3d-dev libopencv-calib3d3.2 libopencv-contrib-dev libopencv-contrib3.2 \
libopencv-core-dev libopencv-core3.2 libopencv-dev libopencv-features2d-dev \
libopencv-features2d3.2 libopencv-flann-dev libopencv-flann3.2 libopencv-highgui-dev \
libopencv-highgui3.2 libopencv-imgcodecs-dev libopencv-imgcodecs3.2 libopencv-imgproc-dev 
RUN apt install -y \
libopencv-imgproc3.2 libopencv-ml-dev libopencv-ml3.2 libopencv-objdetect-dev libopencv-objdetect3.2 libopencv-photo-dev \
libopencv-photo3.2 libopencv-shape-dev libopencv-shape3.2 libopencv-stitching-dev libopencv-stitching3.2 libopencv-superres-dev \
libopencv-superres3.2 libopencv-ts-dev libopencv-video-dev libopencv-video3.2 libopencv-videoio-dev libopencv-videoio3.2 libopencv-videostab-dev \ libopencv-videostab3.2 libopencv-viz-dev libopencv-viz3.2 \
libchromaprint-dev gnutls-bin libgnutls-openssl27 libgnutls28-dev libgnutls30 libnettle6 nettle-dev \
libhogweed4 locate nano 
# && apt-get clean
RUN cd ffmpeg-${FFMPEG_VERSION} && \
./configuration --prefix=/usr \
                --extra-version=0ubuntu0.18.04.1 \
                --toolchain=hardened \
                --libdir=/usr/lib/x86_64-linux-gnu \
                --incdir=/usr/include/x86_64-linux-gnu \
                --enable-gpl \
                --disable-stripping \
                --enable-avresample \
                --enable-avisynth \
                --enable-gnutls \
                --enable-ladspa \
                --enable-libass \
                --enable-libbluray \
                --enable-libbs2b \
                --enable-libcaca \
                --enable-libcdio \
                --enable-libflite \
                --enable-libfontconfig \
                --enable-libfreetype \
                --enable-libfribidi \
                --enable-libgme \
                --enable-libgsm \
                --enable-libmp3lame \
                --enable-libmysofa \
                --enable-libopenjpeg \
                --enable-libopenmpt \
                --enable-libopus \
                --enable-libpulse \
                --enable-librubberband \
                --enable-librsvg \
                --enable-libshine \
                --enable-libsnappy \
                --enable-libsoxr \
                --enable-libspeex \
                --enable-libssh \
                --enable-libtheora \
                --enable-libtwolame \
                --enable-libvorbis \
                --enable-libvpx \
                --enable-libwavpack \
                --enable-libwebp \
                --enable-libx265 \
                --enable-libxml2 \
                --enable-libxvid \
                --enable-libzmq \
                --enable-libzvbi \
                --enable-omx \
                --enable-openal \
                --enable-opengl \
                --enable-sdl2 \
                --enable-libdc1394 \
                --enable-libdrm \
                --enable-libiec61883 \
                --enable-chromaprint \
                --enable-frei0r \
                --enable-libopencv \
                --enable-libx264 \
                --enable-shared 

RUN           make && \
              make install && \
              make distclean && \
              hash -r && \
              rm -rf ${DIR}
#https://github.com/opsxcq/docker-vnc/blob/master/Dockerfile
#RUN apt-get purge -y autoconf automake gcc build-essential git libtool make nasm zlib1g-dev curl
#RUN apt-get clean
#RUN apt-get autoclean
# Create and configure the VNC user
ARG VNCPASS
ENV VNCPASS ${VNCPASS:-secret}
RUN useradd remote --create-home --shell /bin/bash --user-group --groups adm,sudo && \
    echo "remote:$VNCPASS" | chpasswd

EXPOSE 80
EXPOSE 5900
COPY main.sh /usr/local/bin/main.sh
RUN echo "/usr/local/lib" > /etc/ld.so.conf.d/libc.conf

CMD           ["--help"]
ENTRYPOINT    ["ffmpeg"]

In [1]:
!ls *.txt

ffmpeg-manual.txt  history2019-1-20.txt  mylist.txt


In [11]:
f = open("history2019-1-20.txt", "r").readlines()
for line in f:
    if "sudo apt install" in line:
        line = line.replace("\n", "")
        line = line.replace("sudo apt install", "") 
        line = line[5:]
        line = line.replace("  ", "")
        line = line.lstrip(' ')
        print line

inetutils-traceroute
python3-numpy python3-matplotlib python3-basemap python3-pygeoip
yasm
frei0r-plugins frei0r-plugins-dev gnome-video-effects-frei0r
libass-dev libass-devel
libass-dev
libdc1394-22-dev
libmp3lame=3.98.3
libmp3lame==3.98.3
libmp3lame-dev libmp3lame0 libsox-fmt-mp3
libtheora-dev libtheora-bin libtheora0
aptlibvorbis-dev
libvorbis-dev
libvpx-dev
libwavpack-dev libwavpack1
libx264-152 libx264-dev
libx265-146 libx265-dev x265
libx265-146 libx265-dev x265
libxvidcore-dev libxvidcore4
gcc-multilib libc0.1-dev
gcc-multilib
libopenjp2-7-dev libopenjp2-7
libopus-dev
--reinstall librtmp-dev librtmp1
libsoxr-dev libsoxr-lsr0 libsoxr0
speex libspeex-dev
libvo-amrwbenc-dev
libwebp-dev libwebp6
libespeak-dev libespeak1
libav-tools
detox
-y expat libexpat1 libexpat1-dev lua-expat lua-expat-dev node-node-expat xlsx2csv
libexpat1 libexpat1-dev
libexpat1-udeb
libagg-dev
ros-melodic-desktop-full
ros-melodic-desktop-full --fix-missing
ruby-railties
libpng-dev libpnglite-dev libpng-tools 

In [ ]:
yasm frei0r-plugins frei0r-plugins-dev gnome-video-effects-frei0r libass-dev libass-devel \
libass-dev libdc1394-22-dev libmp3lame-dev libmp3lame0 libsox-fmt-mp3 libtheora-dev \
libtheora-bin libtheora0 libvorbis-dev libvpx-dev libwavpack-dev libwavpack1 \
libx264-152 libx264-dev libx265-146 libx265-dev x265 libx265-146 libx265-dev x265 \
libxvidcore-dev libxvidcore4 libopenjp2-7-dev libopenjp2-7 libopus-dev librtmp-dev librtmp1 \
libsoxr-dev libsoxr-lsr0 libsoxr0 speex libspeex-dev libvo-amrwbenc-dev libwebp-dev libwebp6 \
libespeak-dev libespeak1 libav-tools python3-pyopencl libclc-dev pulseaudio pulseaudio-dev \
youtube-dl winff libavcodec-extra libavcodec-extra57 libvo-aacenc-dev vlc-plugin-access-extra \
meld rubberband-ladspa libiec61883-dev ladspa-sdk libopencv-apps-dev libopencv-apps0d \
libopencv-calib3d-dev libopencv-calib3d3.2 libopencv-contrib-dev libopencv-contrib3.2 \
libopencv-core-dev libopencv-core3.2 libopencv-dev libopencv-features2d-dev \
libopencv-features2d3.2 libopencv-flann-dev libopencv-flann3.2 libopencv-highgui-dev \
libopencv-highgui3.2 libopencv-imgcodecs-dev libopencv-imgcodecs3.2 libopencv-imgproc-dev \
libopencv-imgproc3.2 libopencv-ml-dev libopencv-ml3.2 libopencv-objdetect-dev libopencv-objdetect3.2 libopencv-photo-dev libopencv-photo3.2 libopencv-shape-dev libopencv-shape3.2 libopencv-stitching-dev libopencv-stitching3.2 libopencv-superres-dev libopencv-superres3.2 libopencv-ts-dev libopencv-video-dev libopencv-video3.2 libopencv-videoio-dev libopencv-videoio3.2 libopencv-videostab-dev libopencv-videostab3.2 libopencv-viz-dev libopencv-viz3.2
libchromaprint-dev gnutls-bin libgnutls-openssl27 libgnutls28-dev libgnutls30 libnettle6 nettle-dev \
libhogweed4 locate nano 